# Spike Gadgets Ephys and Video Syncing

In [17]:
import re
import sys
from collections import defaultdict
import random
from random import randrange
import glob
import warnings
import os
import git
import bisect


In [2]:
import numpy as np
import pandas as pd
import cv2
from IPython.display import Video
import matplotlib.pyplot as plt

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
import trodes.read_exported

In [5]:
%matplotlib inline

In [6]:
np.random.seed(seed=42)

# Part 0: Index of all the column names

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

# Part 1: Inputting Data

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [7]:
protocol_name = "rc_extention"

## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [8]:
input_folder = "./data"

- Make this cell into non-code block if you are using the same file path for multiple runs

In [9]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [10]:
git_root

'/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls'

In [11]:
recording_filepath_glob = "data/test"

In [12]:
recording_filepath_glob = "data/good/*"

In [13]:
recording_absolute_path_glob = os.path.join(git_root, recording_filepath_glob)

In [14]:
# Getting all the file paths of the recording files(that happen to all end in `.rec`)
raw_data_all_files = glob.glob(recording_absolute_path_glob, recursive=True)

In [15]:
raw_data_all_files

['/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_152723_competition_subject_6_1_and_6_2.rec',
 '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221123_113957_omission_subject_6_1_and_6_4.rec',
 '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221203_154800_omission_and_competition_subject_6_4_and_6_1.rec',
 '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_144832_omission_subject_6_1_and_6_2.rec',
 '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221215_145401_comp_amd_om_6_1_and_6_3.rec',
 '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221122_161341_omission_subject_6_1_and_6_3.rec',
 '/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/2

## Extracting the data and the metadata from the Recording folder

- Creating a dictionary that has the directory as the key and a dictionary that has the file name as the key and the 

In [18]:

def find_closest(my_list, my_number):
    """
    Assumes my_list is sorted. Returns the closest value to my_number.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect.bisect_left(my_list, my_number)
    if pos == 0:
        return my_list[0]
    else:
        return my_list[pos - 1]

In [24]:
raw_data_dir_to_extracted_files = defaultdict(dict)
for raw_data_file_path in raw_data_all_files:
    # Getting the basename of the recording
    recording_dirname = os.path.basename(raw_data_file_path)
    recording_basename = os.path.splitext(recording_dirname)[0]
    # Extracting the files
    raw_data_dir_to_extracted_files[recording_basename]["file_to_data"] = trodes.read_exported.get_all_trodes_data_from_directory(raw_data_file_path)

    # Getting all keys that have "DIO" in it
    # Which is the files that have MED-PC inputs
    DIO_directory_name = trodes.read_exported.get_key_with_substring(raw_data_dir_to_extracted_files[recording_basename]["file_to_data"], substring="DIO")
    raw_data_dir_to_extracted_files[recording_basename]["DIO_directory_dict"] = raw_data_dir_to_extracted_files[recording_basename]["file_to_data"][DIO_directory_name]
    
    # Getting the file that has the timestamps of the tone
    raw_data_dir_to_extracted_files[recording_basename]["tone_onset_DIN_file_name"] = \
        tone_onset_DIN_file_name = trodes.read_exported.get_key_with_substring(raw_data_dir_to_extracted_files[recording_basename]["DIO_directory_dict"] , \
        substring="ECU_Din1.dat", return_first=True)
    
    # Getting the timestamps of the tone
    raw_data_dir_to_extracted_files[recording_basename]["tone_onset_DIN_state_array"] = raw_data_dir_to_extracted_files[recording_basename]["DIO_directory_dict"][tone_onset_DIN_file_name]["data"]
    # Creating a dataframe based on tone times
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"] = pd.DataFrame(raw_data_dir_to_extracted_files[recording_basename]["tone_onset_DIN_state_array"])
    
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"].add_prefix("tone_")
    
    for col in raw_data_dir_to_extracted_files[recording_basename]["tone_df"].columns:
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"][col] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"][col].astype(int)
    
    """
    try:
        # Getting the file that has the box 1 port entries
        raw_data_dir_to_extracted_files[recording_basename]["box1_port_entries_DIN_file_name"] = \
            box1_port_entries_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, \
            substring="ECU_Din3.dat", return_first=True)
        # Getting the timestamps of box 1 port entries
        raw_data_dir_to_extracted_files[recording_basename]["box1_port_entries_DIN_state_array"] = DIO_directory_dict[box1_port_entries_DIN_file_name]["data"]
    except Exception as e: 
        print(e)
        
        
    try:
        # Getting the file that has the box 2 port entries
        raw_data_dir_to_extracted_files[recording_basename]["box2_port_entries_DIN_file_name"] = \
            box2_port_entries_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, \
            substring="ECU_Din1.dat", return_first=True)
        # Getting the timestamps of box 2 port entries
        raw_data_dir_to_extracted_files[recording_basename]["box2_port_entries_DIN_state_array"] = DIO_directory_dict[box2_port_entries_DIN_file_name]["data"]
    except Exception as e: 
        print(e)
    """

    # Getting the timestamps of the voltage samples
    raw_data_dir_to_extracted_files[recording_basename]["raw_directory_dict"] = raw_directory_dict = raw_data_dir_to_extracted_files[recording_basename]["file_to_data"]["raw"]
    raw_data_dir_to_extracted_files[recording_basename]["voltage_timestamp_array"] = raw_directory_dict["timestamps.dat"]["data"]
    raw_data_dir_to_extracted_files[recording_basename]["voltage_timestamp_array"] = raw_data_dir_to_extracted_files[recording_basename]["voltage_timestamp_array"].astype(int)
    
    # Reindexing the voltage timestamps so that the first voltage sample is 0
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["reindexed_timestamp"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["tone_time"] - raw_data_dir_to_extracted_files[recording_basename]["voltage_timestamp_array"][0]
    # Making all the negative values into 0
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["reindexed_timestamp"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["reindexed_timestamp"].apply(lambda x: 0 if x <= 0 else x)
    
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["lfp_index"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["reindexed_timestamp"] // 20

    
    ### VIDEO
    try:
        # read video from file
        video_file_path = glob.glob(os.path.join(raw_data_file_path, "*.fixed.mp4"))[0]
        cap = cv2.VideoCapture(video_file_path)
        # getting the fps from the video
        raw_data_dir_to_extracted_files[recording_basename]["fps"] = cap.get(cv2.CAP_PROP_FPS)
    except Exception as e: 
        print(e)
            
    # Getting the dictionary that containts the timestamps for each videoframe
    video_time_stamp_dict = raw_data_dir_to_extracted_files[recording_basename]["file_to_data"]["."]["1.videoTimeStamps.cameraHWSync"]
    # Getting the timestamps for each frame of the video
    raw_data_dir_to_extracted_files[recording_basename]["PosTimestamp"] = video_time_stamp_dict["data"]["PosTimestamp"].astype(int)
    # Reindexing the video timestamps based on the reindexed voltage timestamp    
    video_time_stamp_array = raw_data_dir_to_extracted_files[recording_basename]["PosTimestamp"] - raw_data_dir_to_extracted_files[recording_basename]["voltage_timestamp_array"][0]
    video_time_stamp_array[video_time_stamp_array <= 0] = 0 
    raw_data_dir_to_extracted_files[recording_basename]["video_time_stamp_array"] = video_time_stamp_array
    
    
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_time_stamp"] = \
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["reindexed_timestamp"].apply(lambda x: find_closest(video_time_stamp_array, x))
    
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_frame"] = \
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_time_stamp"].apply(lambda x: np.where(video_time_stamp_array == x)[0][0])
    
    if "fps" in raw_data_dir_to_extracted_files[recording_basename]:
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_seconds"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_frame"] / raw_data_dir_to_extracted_files[recording_basename]["fps"]
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_seconds"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_seconds"].astype(int)
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_time"] = raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_seconds"].apply(lambda x: divmod(x, 60))
        raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["video_file_path"] = video_file_path
    
    
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["recording_basename"] = recording_basename
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"]["raw_data_file_path"] = raw_data_file_path
    raw_data_dir_to_extracted_files[recording_basename]["tone_df"].to_csv("./proc/timestamp_df/{}.tone_timestamps.csv".format(recording_basename))

/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_152723_competition_subject_6_1_and_6_2.rec/20221125_152723_competition_subject_6_1_and_6_2.rec
  warnings.warn("Can not process {}".format(absolute_file_path))
/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_152723_competition_subject_6_1_and_6_2.rec/20221125_152723_competition_subject_6_1_and_6_2.1.videoTimeStamps
  warnings.warn("Can not process {}".format(absolute_file_path))
/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls

file prefix: 1.videoTimeStamps.cameraHWSync
directory prefix: .
file prefix: analog_ECU_Ain4.dat
directory prefix: analog
file prefix: analog_ECU_Ain3.dat
directory prefix: analog
file prefix: analog_Headstage_GyroZ.dat
directory prefix: analog
file prefix: analog_ECU_Ain6.dat
directory prefix: analog
file prefix: analog_ECU_Ain1.dat
directory prefix: analog
file prefix: analog_ECU_Ain7.dat
directory prefix: analog
file prefix: analog_Controller_Ain1.dat
directory prefix: analog
file prefix: timestamps.dat
directory prefix: analog
file prefix: analog_Headstage_AccelX.dat
directory prefix: analog
file prefix: analog_ECU_Aout4.dat
directory prefix: analog
file prefix: analog_Headstage_AccelZ.dat
directory prefix: analog
file prefix: analog_Headstage_AccelY.dat
directory prefix: analog
file prefix: analog_ECU_Aout1.dat
directory prefix: analog
file prefix: analog_Headstage_MagY.dat
directory prefix: analog
file prefix: analog_ECU_Ain2.dat
directory prefix: analog
file prefix: analog_Heads

/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_152723_competition_subject_6_1_and_6_2.rec/20221125_152723_competition_subject_6_1_top_3_base_2_merged.mountainsort/20221125_152723_competition_subject_6_1_top_3_base_2_merged.timestamps.mda
  warnings.warn("Can not process {}".format(absolute_file_path))
/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_152723_competition_subject_6_1_and_6_2.rec/20221125_152723_competition_subject_6_1_top_3_base_2_merged.mountainsort/20221125_152723_competition_subject_6_1_top_3_

file prefix: spikes_nt30.dat
directory prefix: spikes
file prefix: spikes_nt15.dat
directory prefix: spikes
file prefix: spikes_nt17.dat
directory prefix: spikes
file prefix: spikes_nt26.dat
directory prefix: spikes
file prefix: spikes_nt19.dat
directory prefix: spikes
file prefix: spikes_nt31.dat
directory prefix: spikes
file prefix: spikes_nt25.dat
directory prefix: spikes
file prefix: spikes_nt22.dat
directory prefix: spikes
file prefix: spikes_nt32.dat
directory prefix: spikes
file prefix: spikes_nt10.dat
directory prefix: spikes
file prefix: spikes_nt28.dat
directory prefix: spikes
file prefix: spikes_nt16.dat
directory prefix: spikes
file prefix: spikes_nt8.dat
directory prefix: spikes
file prefix: spikes_nt9.dat
directory prefix: spikes
file prefix: spikes_nt18.dat
directory prefix: spikes
file prefix: spikes_nt3.dat
directory prefix: spikes
file prefix: spikes_nt21.dat
directory prefix: spikes
file prefix: spikes_nt6.dat
directory prefix: spikes
file prefix: spikes_nt1.dat
dire

/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221125_152723_competition_subject_6_1_and_6_2.rec/20221125_152723_competition_subject_6_1_top_3_base_2_merged.kilosort/20221125_152723_competition_subject_6_1_top_3_base_2_merged.group0.dat
  warnings.warn("Can not process {}".format(absolute_file_path))


...

/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221123_122652_competition_subject_6_1_and_6_4.rec/20221123_122652_competition_subject_6_1_top_3_base_3.timestampoffset/20221123_122652_competition_subject_6_1_top_3_base_3.timestampoffset.txt
  warnings.warn("Can not process {}".format(absolute_file_path))
/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221123_122652_competition_subject_6_1_and_6_4.rec/20221123_122652_competition_subject_6_1_top_3_base_3.mountainsort/20221123_122652_competition_subject_6_1_top_3_base_3.timest

file prefix: LFP_nt30ch1.dat
directory prefix: LFP
file prefix: LFP_nt23ch1.dat
directory prefix: LFP
file prefix: LFP_nt3ch1.dat
directory prefix: LFP
file prefix: LFP_nt20ch1.dat
directory prefix: LFP
file prefix: LFP_nt11ch1.dat
directory prefix: LFP
file prefix: LFP_nt16ch1.dat
directory prefix: LFP
file prefix: LFP_nt13ch1.dat
directory prefix: LFP
file prefix: group0.coordinates.dat
directory prefix: mountainsort
file prefix: group0.coordinates.dat
directory prefix: kilosort
file prefix: timestamps.dat
directory prefix: kilosort


/nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/results/2023_05_17_sleap_analysis/../../src/trodes/read_exported.py:138: UserWarning: Can not process /nancy/projects/reward_competition_ephys_analysis_with_omission_and_divider_controls/data/good/20221123_122652_competition_subject_6_1_and_6_4.rec/20221123_122652_competition_subject_6_1_top_3_base_3.kilosort/20221123_122652_competition_subject_6_1_top_3_base_3.group0.dat
  warnings.warn("Can not process {}".format(absolute_file_path))


file prefix: timestamps.dat
directory prefix: time


KeyError: '1.videoTimeStamps.cameraHWSync'

In [25]:
raw_data_dir_to_extracted_files[recording_basename]["tone_df"]

,tone_time,tone_state,reindexed_timestamp,lfp_index
0,2253424,1,0,0
1,2474955,0,221531,11076
2,3675351,1,1421927,71096
3,3875353,0,1621929,81096
4,5275349,1,3021925,151096
5,5475348,0,3221924,161096
6,7075341,1,4821917,241095
7,7275341,0,5021917,251095


In [27]:
recording_basename

'20221123_122652_competition_subject_6_1_and_6_4'

In [26]:
raise ValueError()

ValueError: 

# Part 2: Looking over the data

## Looking over the ephys recording

- Getting the name of the ephys recording directory

In [ ]:
raw_directory_dict = file_to_data["raw"]

- Getting the name of the files in the ephys recording directory

In [ ]:
raw_directory_dict.keys()

In [ ]:
raw_recording_fields_text = raw_directory_dict["raw_group0.dat"]

In [ ]:
raw_recording_fields_text

- Array of Voltages for each channel

In [ ]:
voltage_value_array = raw_recording_fields_text["data"]

In [ ]:
voltage_value_array.shape

In [ ]:
voltage_value_array[0]

In [ ]:
len(voltage_value_array[0][0])

In [ ]:
voltage_value_array[:5]

- Voltage Time Stamps

In [ ]:
raw_directory_dict.keys()

In [ ]:
voltage_timestamp_array = raw_directory_dict["timestamps.dat"]["data"]

In [ ]:
voltage_timestamp_array[:5]

In [ ]:
voltage_timestamp_array[-5:]

In [ ]:
voltage_timestamp_array.shape

- Converting the array to integers to be able to do calculations

    - u4 and i2 explanation: https://www.geeksforgeeks.org/data-type-object-dtype-numpy-python/

In [ ]:
voltage_timestamp_array = voltage_timestamp_array.astype(int)

In [ ]:
voltage_timestamp_array[:5]

In [ ]:
voltage_timestamp_array[-5:]

In [ ]:
voltage_timestamp_array.shape

## Looking over the video files

In [ ]:
parent_directory_dict = file_to_data["."]

In [ ]:
video_time_stamp_dict = parent_directory_dict["1.videoTimeStamps.cameraHWSync"]

In [ ]:
video_time_stamp_dict

In [ ]:
video_time_stamp_dict["data"]

In [ ]:
video_time_stamp_dict["data"][:5]

In [ ]:
video_time_stamp_dict["data"][-5:]

In [ ]:
video_time_stamp_dict["data"].shape

## Video time to Video frame

### Reading in video

In [ ]:
raw_data_file_path

In [ ]:
video_file_list = glob.glob(os.path.join("../../../../data/good/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec/", "*.mp4"))

In [ ]:
video_file_list

In [ ]:
video_file_path = video_file_list[0]

In [ ]:
# read video from file
cap = cv2.VideoCapture(video_file_path)

- Getting the number of frames per second

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [ ]:
fps

- Total number of frames

In [ ]:
total_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [ ]:
total_frame_count


- Calculating the length of the video by dividing the total number of frames by the fps

In [ ]:
video_length = total_frame_count / fps

In [ ]:
video_length

# Part 3: Converting everything into timestamps

## Voltage to timestamp

- Use the matching index to convert between the voltage time stamp and the recording

In [ ]:
voltage_index = 0

In [ ]:
voltage_value_array[voltage_index]

- Getting the associated timestamp of the ephys recording

In [ ]:
voltage_timestamp_array[:5]

In [ ]:
voltage_timestamp_array[-5:]

In [ ]:
voltage_time_stamp = voltage_timestamp_array[voltage_index]

In [ ]:
voltage_time_stamp

## Video frame to timestamp

- Getting the time stamps of the video frames
    - Each frame would correspond to each timestamp. Because the sample rate of videos is smaller than ephys recording, the number of video time stamps will be less than that of ephys recordings.

In [ ]:
video_time_stamp_dict

In [ ]:
video_time_stamp_array = np.array(video_time_stamp_dict["data"])

In [ ]:
video_time_stamp_array[:5]

In [ ]:
video_time_stamp_array[-5:]

In [ ]:
video_time_stamp_array.shape

- Getting only the first number in each tuple

In [ ]:
pos_timestamp_array = np.array([x[0] for x in video_time_stamp_array]) 

- Converting to integer to do calculations

In [ ]:
pos_timestamp_array = pos_timestamp_array.astype(int)

In [ ]:
pos_timestamp_array[:5]

In [ ]:
pos_timestamp_array[-5:]

# Part 4: Converting from timestamps back to everything

In [ ]:
def timestamp_to_index(current_ts, ts_array):
    """
    """
    return np.argwhere(ts_array >= current_ts)[0][0]

## Time stamp to Voltage Value

In [ ]:
voltage_timestamp_array

In [ ]:
voltage_timestamp_array[0]

- Getting the index of the closest timestamp 

In [ ]:
random_voltage_index = timestamp_to_index(current_ts=voltage_timestamp_array[0], ts_array=voltage_timestamp_array)

In [ ]:
random_voltage_index

- Checking if it matches to the original timestamp
    - Should be the same, because the sampling rate of the timestamps are based on the ephys recording

In [ ]:
voltage_timestamp_array[random_voltage_index]

- Getting the voltage value based on the index

In [ ]:
voltage_value_array[random_voltage_index]

## Time Stamp to Video Frame

In [ ]:
video_time_stamp_array

In [ ]:
pos_timestamp_array

- Getting the index of the closest timestamp. The index corresponds to the video's frame number

In [ ]:
random_video_frame = timestamp_to_index(current_ts=pos_timestamp_array[0], ts_array=pos_timestamp_array)

In [ ]:
random_video_frame

In [ ]:
pos_timestamp_array[random_video_frame]

# ADDED AFTER

# Part 4: Syncing everything based on timestamps

# Syncing with MED-PC

## Looking over the MED-PC Data

- Box 1 Port Entries
    - ECU Din3
- Box 2 Port Entries
    - Controller Din1
- Box 1 Tone playing
    - ECU Din1

In [ ]:
DIO_directory_name = trodes.read_exported.get_key_with_substring(file_to_data, substring="DIO")

In [ ]:
DIO_directory_name

In [ ]:
DIO_directory_dict = file_to_data[DIO_directory_name]

In [ ]:
DIO_directory_dict.keys()

### Tone Onset Signal

In [ ]:
tone_onset_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, substring="ECU_Din1.dat", return_first=True)

In [ ]:
tone_onset_DIN_file_name

In [ ]:
tone_onset_DIN_state_array = DIO_directory_dict[tone_onset_DIN_file_name]["data"]

In [ ]:
tone_onset_DIN_state_array

In [ ]:
plt.hist([tup[1] for tup in tone_onset_DIN_state_array])

In [ ]:
plt.plot([tup[0] for tup in tone_onset_DIN_state_array], [tup[1] for tup in tone_onset_DIN_state_array])
plt.xlabel("Timestamp")
plt.ylabel("State")
plt.title("Din State Change against Timestamps")

### Box 1 Port Entries

In [ ]:
box1_port_entries_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, substring="ECU_Din3.dat", return_first=True)

In [ ]:
box1_port_entries_DIN_file_name

In [ ]:
box1_port_entries_DIN_state_array = DIO_directory_dict[box1_port_entries_DIN_file_name]["data"]

In [ ]:
box1_port_entries_DIN_state_array

In [ ]:
plt.hist([tup[1] for tup in box1_port_entries_DIN_state_array])

- There is a gap in the middle when the protocol was being changed between competition or omission

In [ ]:
plt.plot([tup[0] for tup in box1_port_entries_DIN_state_array], [tup[1] for tup in box1_port_entries_DIN_state_array])
plt.xlabel("Timestamp")
plt.ylabel("State")
plt.title("Din State Change against Timestamps")

### Box 2 Port Entries

In [ ]:
box2_port_entries_DIN_file_name = trodes.read_exported.get_key_with_substring(DIO_directory_dict, substring="dio_Controller_Din1.dat", return_first=True)

In [ ]:
box2_port_entries_DIN_file_name

In [ ]:
box2_port_entries_DIN_file_name = DIO_directory_dict[box2_port_entries_DIN_file_name]["data"]

In [ ]:
box2_port_entries_DIN_file_name

In [ ]:
plt.hist([tup[1] for tup in box2_port_entries_DIN_file_name])

- This is half the time than the previous port entries, because the mouse was moved to box 1 for half of the session

In [ ]:
plt.plot([tup[0] for tup in box2_port_entries_DIN_file_name], [tup[1] for tup in box2_port_entries_DIN_file_name])
plt.xlabel("Timestamp")
plt.ylabel("State")
plt.title("Din State Change against Timestamps")

## Labeling the Tone and Port Entries

# Syncing with MED-PC

- List of when the ECU has changed signal. 1 means that the ECU Din1 signal is on, 0 means it's off.

In [ ]:
tone_onset_DIN_state_array

- Checking to see if 1 or 0 is when the tone plays
    - Dividing by 20000, because we are recording at a sampling rate at 20000

In [ ]:
tone_onset_DIN_state_array[0][0]

In [ ]:
tone_onset_DIN_state_array[0][1]

In [ ]:
tone_onset_DIN_state_array[1][0]

In [ ]:
tone_onset_DIN_state_array[1][1]

In [ ]:
tone_onset_DIN_state_array[2][0]

In [ ]:
tone_onset_DIN_state_array[2][1]

- So the tone starts when the state is "1"
    - This can be seen because the time from 1 to 0 is less than 60 seconds

In [ ]:
first_delay = (tone_onset_DIN_state_array[1][0] - tone_onset_DIN_state_array[0][0]) / 20000

In [ ]:
first_delay

- Time difference for on >>> off

In [ ]:
(tone_onset_DIN_state_array[3][0] - tone_onset_DIN_state_array[2][0]) / 20000

- So the tone starts when the state is "2"
    - This can be seen because the time from 0 to 1 is 60 seconds, the time for one session

In [ ]:
(tone_onset_DIN_state_array[2][0] - tone_onset_DIN_state_array[1][0]) / 20000

- Getting only the times when the ECU signal was on

In [ ]:
tone_din_time = [din_time for din_time, din_state in tone_onset_DIN_state_array if din_state == 1]

In [ ]:
len(tone_din_time)

In [ ]:
tone_din_time[:10]

# From DIN to Video

- State 1 is when the MED-PC signal is being recieved. And 0 is when it is turned off. So we will get the timestamp of when it is first 1.

In [ ]:
tone_onset_DIN_state_array

In [ ]:
tone_time_stamp = [time_state[0] for time_state in tone_onset_DIN_state_array if time_state[1]]

In [ ]:
example_tone_time_stamp = tone_time_stamp[3]

In [ ]:
example_tone_time_stamp

- Array of the time stamp of all the frames

In [ ]:
pos_timestamp_array

- Getting the first video time stamp that is greater than the voltage time stamp

In [ ]:
current_video_frame = timestamp_to_index(current_ts=example_tone_time_stamp, ts_array=pos_timestamp_array)

In [ ]:
current_video_frame

In [ ]:
timestamp_to_index(current_ts=28625643, ts_array=pos_timestamp_array)

# Syncing up the timestamps using Pandas

## Adding the Voltage as columns

In [ ]:
voltage_timestamp_array[:5]

In [ ]:
voltage_timestamp_array.shape

In [ ]:
voltage_value_array[:5]

In [ ]:
voltage_value_array.shape

- Adding the voltage timestamps

In [ ]:
ephys_dataframe = pd.DataFrame(voltage_timestamp_array, columns=["voltage_time_stamp"])

In [ ]:
ephys_dataframe.head()

- Adding the voltage value

In [ ]:
ephys_dataframe.head()

## Adding the video data as columns

- Creating a seperate dataframe for video data first

In [ ]:
pos_timestamp_array[:5]

In [ ]:
pos_timestamp_array[-5:]

In [ ]:
video_dataframe = pd.DataFrame(pos_timestamp_array, columns=["PosTimestamp"])

In [ ]:
video_dataframe.head()

- Adding the frames which would just be the number in the list that the timestamps belongs to

In [ ]:
video_dataframe.insert(0, 'frame_number', range(1, 1 + len(video_dataframe)))

- Calculating the time within the video by dividing the frame by the fps

In [ ]:
video_dataframe["video_time"] = video_dataframe["frame_number"] / fps

In [ ]:
video_dataframe["video_seconds"] = video_dataframe["video_time"].astype(int)

In [ ]:
video_dataframe

## Combining the ephys and video dataframe into one

In [ ]:
ephy_and_video_dataframe = pd.merge(ephys_dataframe, video_dataframe, left_on='voltage_time_stamp', right_on='PosTimestamp', how="left")

In [ ]:
ephy_and_video_dataframe

In [ ]:
ephy_and_video_dataframe.columns

- There are only a small number of rows that have information for the video, because the sampling rate is much smaller. 

In [ ]:
ephy_and_video_dataframe.dropna(subset=["PosTimestamp"])

- Filling in all the blank cells with the previous rows for the video related columns into new columns. This can be used to select for all rows that correspond to something happening within the video

In [ ]:
ephy_and_video_dataframe.columns

In [ ]:
for col in ephy_and_video_dataframe.columns:
    if "filled" not in col:
        ephy_and_video_dataframe['filled_{}'.format(col)] = ephy_and_video_dataframe[col].fillna(method='ffill')


In [ ]:
ephy_and_video_dataframe['filled_PosTimestamp'] = ephy_and_video_dataframe['PosTimestamp'].fillna(method='ffill')
ephy_and_video_dataframe['filled_frame_number'] = ephy_and_video_dataframe['frame_number'].fillna(method='ffill')
ephy_and_video_dataframe['filled_video_time'] = ephy_and_video_dataframe['video_time'].fillna(method='ffill')
ephy_and_video_dataframe['filled_video_seconds'] = ephy_and_video_dataframe['video_seconds'].fillna(method='ffill')

In [ ]:
ephy_and_video_dataframe.tail()

## Adding the DIN info

In [ ]:
DIN_dataframe = pd.DataFrame(tone_onset_DIN_state_array)

- Dropping the first two rows because that is when things start

In [ ]:
DIN_dataframe = DIN_dataframe.drop([0,1]).reset_index(drop=True)

In [ ]:
DIN_dataframe["trial_number"] = DIN_dataframe["state"].cumsum()

In [ ]:
DIN_dataframe

In [ ]:
ephy_and_video_dataframe = pd.merge(ephy_and_video_dataframe, DIN_dataframe, left_on='voltage_time_stamp', right_on='time', how="left")


In [ ]:
ephy_and_video_dataframe.head()

In [ ]:
tone_info_dataframe = ephy_and_video_dataframe.dropna(subset=["time"]).reset_index(drop=True)


In [ ]:
tone_info_dataframe["voltage_index"] = tone_info_dataframe["voltage_time_stamp"] - voltage_timestamp_array[0]

In [ ]:
tone_info_dataframe = tone_info_dataframe.dropna(axis="columns")
# To remove last tone light
tone_info_dataframe = tone_info_dataframe.iloc[0:-1]

In [ ]:
tone_info_dataframe

In [ ]:
results_dict = {1390826: 'rewarded',
 2990825: 'rewarded',
 4790823: 'rewarded',
 6390821: 'omission',
 7890820: 'rewarded',
 9890818: 'rewarded',
 11790816: 'rewarded',
 13590815: 'rewarded',
 15190821: 'omission',
 16990812: 'rewarded',
 18990809: 'rewarded',
 20790805: 'omission',
 23190805: 'rewarded',
 24990804: 'rewarded',
 30949197: 'win',
 32549196: 'win',
 34349195: 'win',
 35949193: 'win',
 37449192: 'win',
 39449187: 'win',
 41349188: 'win',
 43149186: 'win',
 44749185: 'win',
 46549183: 'win',
 48549181: 'win',
 50349180: 'win',
 52749175: 'win',
 54549173: 'win',
 56249171: 'win',
 58049170: 'win',
 59949171: 'win',
 62349168: 'win',
 63949167: 'win'}

In [ ]:
tone_info_dataframe["trial_type"] = tone_info_dataframe["voltage_index"].map(results_dict)

In [ ]:
tone_info_dataframe

In [ ]:
tone_info_dataframe[tone_info_dataframe["trial_type"] == "rewarded"]

In [ ]:
tone_info_dataframe[tone_info_dataframe["trial_type"] == "win"]

In [ ]:
recording_file_name = file_to_data["raw"]["raw_group0.dat"]["original_file"]

In [ ]:
recording_base_name = os.path.splitext(recording_file_name)[0]

In [ ]:
recording_base_name

In [ ]:
tone_info_dataframe.to_csv("./proc/{}.tone_timestamps.csv".format(recording_base_name))

In [ ]:
tone_info_dataframe

In [ ]:
raise ValueError()

In [ ]:
ephy_and_video_dataframe["filled_state"] = ephy_and_video_dataframe["state"].ffill()
ephy_and_video_dataframe["filled_trial_number"] = ephy_and_video_dataframe["trial_number"].ffill()

In [ ]:
ephy_and_video_dataframe.head()

In [ ]:
ephy_and_video_dataframe.tail()

In [ ]:
trial_1_df = ephy_and_video_dataframe[ephy_and_video_dataframe["filled_trial_number"] == 2]

In [ ]:
trial_1_df.head()

In [ ]:
file_to_data["raw"]

In [ ]:
ephy_and_video_dataframe.to_csv("./proc/{}.timestamps.csv".format(recording_base_name))

In [ ]:
raise ValueError()

- Original frame number(before light turns on)

In [ ]:
current_video_frame = trial_1_df["filled_frame_number"].min()

- Corrected frame number(that has the light on)

In [ ]:
corrected_video_frame = current_video_frame + 2

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, corrected_video_frame)


In [ ]:
_, frame = cap.read()

In [ ]:
video_file_path

In [ ]:
output_directory = "./proc"

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
video_file_basename = os.path.basename(video_file_path)

In [ ]:
video_file_root = os.path.splitext(video_file_basename)[0]

In [ ]:
video_file_root

In [ ]:
cv2.imwrite(os.path.join(output_directory, '{}.frame_{}.png'.format(video_file_root, corrected_video_frame)), frame)

# OTHER STUFF

## Getting the specific frame

In [ ]:
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)

- Original frame number(before light turns on)

In [ ]:
corrected_video_frame = current_video_frame

- Corrected frame number(that has the light on)

In [ ]:
corrected_video_frame = current_video_frame + 2

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, corrected_video_frame)


In [ ]:
_, frame = cap.read()

In [ ]:
video_file_path

In [ ]:
output_directory = "./proc"

In [ ]:
output_directory

In [ ]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
video_file_basename = os.path.basename(video_file_path)

In [ ]:
video_file_root = os.path.splitext(video_file_basename)[0]

In [ ]:
video_file_root

In [ ]:
cv2.imwrite(os.path.join(output_directory, '{}.frame_{}.png'.format(video_file_root, corrected_video_frame)), frame)

- Where this video time stamp is(within the list of video time stamps), would be the frame number that corresponds to the ephys recording instance

In [ ]:
current_video_seconds = corrected_video_frame / fps

In [ ]:
current_video_seconds

In [ ]:
print("MED-PC signal is at {}:{}".format(int(current_video_seconds // 60), int(current_video_seconds % 60)))